# Lower Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ x := L x $ where $ L $ is lower triangular.  Vector $ y $ is not to be touched (and, indeed, not even passed into the routines).  This is a little trickier than you might think.  Order matters, since you don't want to corrupt values of $ x $ that are still needed.  Indeed, you may want to do a few small problems by hand if you don't get the right answer.  Also, PictureFLAME may help you see what is going on.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

## The routine <br> <code> Trmv_ln_unb_var1( L, x ) </code>

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := L x $.  The "_un_" in the name of the routine indicates this is the "lower, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.scal( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

<h1> <font color=red> NOTE: The algorithms for this operation march from bottom-right to top-left!!! </font> </h1>

In [1]:
import flame
import laff as laff

def Trmv_ln_unb_var1(L, x):

    LTL, LTR, \
    LBL, LBR  = flame.part_2x2(L, \
                               0, 0, 'BR')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'BOTTOM')

    while LBR.shape[0] < L.shape[0]:

        L00,  l01,      L02,  \
        l10t, lambda11, l12t, \
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, \
                                                        LBL, LBR, \
                                                        1, 1, 'TL')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'TOP')

        #------------------------------------------------------------#

        laff.scal(lambda11, chi1)
        laff.dots(l10t, x0, chi1)

        #------------------------------------------------------------#

        LTL, LTR, \
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  \
                                               l10t, lambda11, l12t, \
                                               L20,  l21,      L22,  \
                                               'BR')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'BOTTOM')

    flame.merge_2x1(xT, \
                    xB, x)


## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

L = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
xold = matrix( random.rand( 4,1 ) )

# Notice that L is not lower triangular.  We will only use the lower triangular part.

print( 'L before =' )
print( L )

print( 'x before =' )
print( x )

L before =
[[0.50565271 0.12811406 0.08186772 0.9000754 ]
 [0.59277258 0.76665009 0.70257588 0.82928213]
 [0.38160516 0.67544731 0.0579909  0.51675956]
 [0.85104374 0.03912859 0.16937466 0.31202438]]
x before =
[[0.19108845]
 [0.82915538]
 [0.16325694]
 [0.38463522]]


In [3]:
import numpy as np
laff.copy( x, xold )   # save the original vector x

Trmv_ln_unb_var1( L, x )

print( 'x after =' )
print( x )

print( np.tril( L ) * xold )

print( 'x - ( np.tril( L ) * xold ) = ' ) #np.tril makes a matrix lower triangular
print( x - ( np.tril( L ) * xold ) )

x after =
[[0.09662439]
 [0.74894404]
 [0.64243852]
 [0.34273546]]
[[0.09662439]
 [0.74894404]
 [0.64243852]
 [0.34273546]]
x - ( np.tril( L ) * xold ) = 
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-5.55111512e-17]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The routine <br> <code> Trmv_ln_unb_var2( L, x ) </code>

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := L x $.  The "_ln_" in the name of the routine indicates this is the "lower triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
<li> <code> laff.scal( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
def Trmv_ln_unb_var2(L, x):

    LTL, LTR, \
    LBL, LBR  = flame.part_2x2(L, \
                               0, 0, 'BR')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'BOTTOM')

    while LBR.shape[0] < L.shape[0]:

        L00,  l01,      L02,  \
        l10t, lambda11, l12t, \
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, \
                                                        LBL, LBR, \
                                                        1, 1, 'TL')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'TOP')

        #------------------------------------------------------------#

        laff.axpy(chi1, l21, x2)
        laff.scal(lambda11, chi1)

        #------------------------------------------------------------#

        LTL, LTR, \
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  \
                                               l10t, lambda11, l12t, \
                                               L20,  l21,      L22,  \
                                               'BR')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'BOTTOM')

    flame.merge_2x1(xT, \
                    xB, x)


## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

L = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
xold = matrix( random.rand( 4,1 ) )

# L is not lower triangular.  We will only use the lower triangular part.

print( 'L before =' )
print( L )

print( 'x before =' )
print( x )


L before =
[[0.94800373 0.42815851 0.12593172 0.90985275]
 [0.58341779 0.68450374 0.12804429 0.39517578]
 [0.56313299 0.98559674 0.92533203 0.3738467 ]
 [0.91612134 0.14218654 0.19798688 0.66021948]]
x before =
[[0.68801217]
 [0.81794587]
 [0.83800253]
 [0.51147769]]


In [6]:
laff.copy( x, xold )   # save the original vector y

Trmv_ln_unb_var2( L, x )

print( 'x after =' )
print( x )

print( 'x - ( np.tril( L ) * xold ) = ' ) #np.tril makes the matrix lower triangular
print( x - ( np.tril( L ) * xold ) )

x after =
[[0.6522381 ]
 [0.96128555]
 [1.96903772]
 [1.25020456]]
x - ( np.tril( L ) * xold ) = 
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-2.22044605e-16]
 [ 0.00000000e+00]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.